In [1]:
# 기본
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from matplotlib import font_manager, rc
import platform
from tqdm import tqdm
import sklearn
from sklearn import linear_model
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
from sklearn.model_selection import train_test_split

# time
import datetime

# crawling
import requests
import lxml.html
import sqlite3
from pandas.io import sql
from bs4 import BeautifulSoup
import json

# 한글 폰트 설정
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)

In [185]:
lst = [1, 2]
for l in lst: 
    print(l)

for i in range(4-len(lst)): 
    lst.append('none')

lst

1
2


[1, 2, 'none', 'none']

In [166]:
# 김시언 박사님 코드 (사용 x)

def parse_lab_results(input_string):
    #preprocessing lines...
    lines = [l for l in input_string.split('\n\n') if l.strip()]
    lines = [l.replace(' (', '(')  for l in lines]
    lines = [l.replace('--', '')  for l in lines]
    
    parsed_results = []
    test_name_previous = 'none'
    i1 = 4
    i2 = 8
    i3 = 33
    i4 = 54
    print(lines)
    for line in lines:
        parts = line.split()
        print(line)
        print(parts)
        
        #get test_name
        #fine-tune here... 3
        if line.index(parts[0])<=i1:
            test_name = parts[0]
        else:
            test_name = test_name_previous
            parts.insert(0, test_name_previous)
        test_name_previous = test_name
        
        #get detailed_test_type
        #fine-tune here... 8, 15, 30
        if i1<line.index(parts[1])<=i2:
            detailed_test_type = parts[1]
        else:    
            detailed_test_type = 'none'
            parts.insert(1, 'none')
            
        #get test_results_value
        #fine-tune here... 8, 15, 30
        if i2<line.index(parts[2])<=i3:
            test_results_value = parts[2]
        else:    
            test_results_value = 'none'
            parts.insert(2, 'none')
            
        #get test_result_unit
        #fine-tune here... 8, 15, 30
        try:
            #get test_result_unit
            #fine-tune here... 8, 15, 30
            if i3<=line.index(parts[3])<=i4:
                test_result_unit = parts[3]
            else:    
                test_result_unit = 'none'
                parts.insert(3, 'none')
        except IndexError:
            test_result_unit = 'none'
            
        #get test_result_unit
        #fine-tune here... 8, 15, 30
        try:
            if i4<line.index(parts[4]):
                doctor_opinion = parts[4]
            else:    
                doctor_opinion = 'none'
                parts.insert(4, 'none')
        except IndexError:
            doctor_opinion = 'none'
            
        parsed_results.append({
            'test_name': [test_name],
            'detailed_test_type': [detailed_test_type],
            'test_results_value': [test_results_value],
            'test_result_unit': [test_result_unit],
            'doctor_opinion': [doctor_opinion]
        })

    return parsed_results


In [193]:
def parse_lab_results_zio(input_string):
    #preprocessing lines...
    lines = [l for l in input_string.split('\n\n') if l.strip()]
    lines = [l.replace(' (', '(')  for l in lines]
    lines = [l.replace('--', '')  for l in lines]
    
    parsed_results = []
    test_name_previous = 'none'

    print(lines)
    for line in lines:
        parts = line.split()
        print(line)
        print(parts)

        for i in range(5-len(parts)): 
            parts.append('none')

        if parts[0] in ['VLDL', 'IDL', 'LDL', 'HDL']: 
            test_name = parts[0]
        elif parts[0] in ['2(large)', '3(small)', '4(small)', '5(small)', '6(small)', '7(small)']: 
            test_name = 'LDL'
        else: 
            test_name = 'none'
            detailed_test_type = 'none'
            test_results_value = 'none'
            test_result_unit = 'none'
            doctor_opinion = 'none'

        if test_name == 'VLDL': 
            detailed_test_type = 'none'
            test_results_value = parts[1]
            test_result_unit = parts[2]
            doctor_opinion = parts[3]

        if test_name == 'IDL': 
            detailed_test_type = parts[1]
            test_results_value = parts[2]
            test_result_unit = parts[3]
            doctor_opinion = 'none'
        
        if test_name == 'LDL': 
            if parts[0] == 'LDL': 
                detailed_test_type = parts[1]
                test_results_value = parts[2]
                test_result_unit = parts[3]
                doctor_opinion = parts[4]
            else: 
                detailed_test_type = parts[0]
                test_results_value = parts[1]
                test_result_unit = parts[2]
                doctor_opinion = parts[3]

        if test_name == 'HDL': 
            detailed_test_type = 'none'
            test_results_value = parts[1]
            test_result_unit = parts[2]
            doctor_opinion = parts[3]

        parsed_results.append({
            'test_name': [test_name],
            'detailed_test_type': [detailed_test_type],
            'test_results_value': [test_results_value],
            'test_result_unit': [test_result_unit],
            'doctor_opinion': [doctor_opinion]
        })

    return parsed_results


In [194]:
data = pd.read_csv("C:\\Users\\admin\\Downloads\\ldlparticle.csv", encoding = 'utf-8')
data

,PT_NO,RID,연구별 환자 ID,환자명,성별,생년월일,병원코드,검사명,검사시행일,검사결과,검사결과-수치값
0,7736588,R-104-00056192,R-104-00056192,위**,F,1935-12,1,[검사중단]LDL particle size 분석(외주),2008-11-25 10:35:09.0,<< LDL subclass and particle size 분석 >>\n\n\n\...,NaN
1,7745708,R-104-00056194,R-104-00056194,박**,F,1945-11,1,[검사중단]LDL particle size 분석(외주),2004-10-05 12:30:00.0,<< LDL subclass and particle size 분석 >>\n\n ...,NaN
2,7749096,R-104-00068156,R-104-00068156,강**,M,1950-11,1,LDL particle size 분석,2018-09-06 09:16:00.0,<< LDL subclass and particle size 분석 >>\n\n■ 검...,NaN
3,7768237,R-104-00085337,R-104-00085337,이**,F,1946-12,1,LDL particle size 분석,2018-12-03 09:37:23.0,<< LDL subclass and particle size 분석 >>\n\n■ 검...,NaN
4,7795741,R-104-00033389,R-104-00033389,소**,F,1940-09,1,LDL particle size 분석,2012-12-04 09:21:06.0,<< LDL subclass and particle size 분석 >>\n\n ...,NaN
...,...,...,...,...,...,...,...,...,...,...,...
6037,71207632,R-104-00032016,R-104-00032016,김**,M,1979-09,1,LDL particle size 분석,2021-06-29 09:04:43.0,<< LDL subclass and particle size 분석 >>\n\n■ 검...,NaN
6038,71332273,R-104-00085218,R-104-00085218,김**,F,1987-01,1,LDL particle size 분석,2010-12-17 10:15:10.0,<< LDL subclass and particle size 분석 >>\n\n ...,NaN
6039,71604598,R-104-00331415,R-104-00331415,신**,F,1982-07,1,LDL particle size 분석,2012-07-06 08:38:01.0,<< LDL subclass and particle size 분석 >>\n\n ...,NaN
6040,73421364,R-104-00332445,R-104-00332445,백**,F,1986-10,1,LDL particle size 분석,2010-11-12 11:09:29.0,<< LDL subclass and particle size 분석 >>\n\n ...,NaN


In [195]:
data2 = data['검사결과'].str.split('-------------------------------------------------------', expand = True)
data2.drop(columns=[0, 1, 3, 4, 5, 6], inplace = True)
data2['검사결과'] = 'none'
data2.rename(columns = {2 : 'lab test results'}, inplace = True)
data2 = pd.concat([data['PT_NO'], data2], axis = 1).copy()
data2

,PT_NO,lab test results,검사결과
0,7736588,\n\n VLDL 5-22 ...,none
1,7745708,\n\n VLDL 5-22 ...,none
2,7749096,--\n VLDL 5-22 ...,none
3,7768237,--\n VLDL 5-22 ...,none
4,7795741,\n\n VLDL 5-22 ...,none
...,...,...,...
6037,71207632,--\n VLDL 5-22 ...,none
6038,71332273,\n\n VLDL 5-22 ...,none
6039,71604598,\n\n VLDL 5-22 ...,none
6040,73421364,\n\n VLDL 5-22 ...,none


In [196]:
parse_lab_results_zio(data2['lab test results'][40])

['    VLDL                     5-22           28      High', '    IDL   C                 11-22           18      ', '    IDL   B                  5-15           10      ', '    IDL   A                  8-25           10      ', '    LDL   1(large)         25-57           23      Low', '          2(large)          7-29           25      ', '          3(small)           0-4           5       High', '          4(small)                         0       ', '          5(small)                         0       ', '          6(small)                         0       ', '          7(small)                         0       ', '    HDL                     40-79           37      Low']
    VLDL                     5-22           28      High
['VLDL', '5-22', '28', 'High']
    IDL   C                 11-22           18      
['IDL', 'C', '11-22', '18']
    IDL   B                  5-15           10      
['IDL', 'B', '5-15', '10']
    IDL   A                  8-25           10      
['IDL', 'A', '8-25

[{'test_name': ['VLDL'],
  'detailed_test_type': ['none'],
  'test_results_value': ['5-22'],
  'test_result_unit': ['28'],
  'doctor_opinion': ['High']},
 {'test_name': ['IDL'],
  'detailed_test_type': ['C'],
  'test_results_value': ['11-22'],
  'test_result_unit': ['18'],
  'doctor_opinion': ['none']},
 {'test_name': ['IDL'],
  'detailed_test_type': ['B'],
  'test_results_value': ['5-15'],
  'test_result_unit': ['10'],
  'doctor_opinion': ['none']},
 {'test_name': ['IDL'],
  'detailed_test_type': ['A'],
  'test_results_value': ['8-25'],
  'test_result_unit': ['10'],
  'doctor_opinion': ['none']},
 {'test_name': ['LDL'],
  'detailed_test_type': ['1(large)'],
  'test_results_value': ['25-57'],
  'test_result_unit': ['23'],
  'doctor_opinion': ['Low']},
 {'test_name': ['LDL'],
  'detailed_test_type': ['2(large)'],
  'test_results_value': ['7-29'],
  'test_result_unit': ['25'],
  'doctor_opinion': ['none']},
 {'test_name': ['LDL'],
  'detailed_test_type': ['3(small)'],
  'test_results_val

In [ ]:
for i in range(len(data2)): 
    try: 
        data2['검사결과'][i] = parse_lab_results_zio(data2['lab test results'][i])
    except IndexError: 
        data2['검사결과'][i] = 'none'

In [198]:
data2

,PT_NO,lab test results,검사결과
0,7736588,\n\n VLDL 5-22 ...,"[{'test_name': ['VLDL'], 'detailed_test_type':..."
1,7745708,\n\n VLDL 5-22 ...,"[{'test_name': ['VLDL'], 'detailed_test_type':..."
2,7749096,--\n VLDL 5-22 ...,"[{'test_name': ['VLDL'], 'detailed_test_type':..."
3,7768237,--\n VLDL 5-22 ...,"[{'test_name': ['VLDL'], 'detailed_test_type':..."
4,7795741,\n\n VLDL 5-22 ...,"[{'test_name': ['VLDL'], 'detailed_test_type':..."
...,...,...,...
6037,71207632,--\n VLDL 5-22 ...,"[{'test_name': ['VLDL'], 'detailed_test_type':..."
6038,71332273,\n\n VLDL 5-22 ...,"[{'test_name': ['VLDL'], 'detailed_test_type':..."
6039,71604598,\n\n VLDL 5-22 ...,"[{'test_name': ['VLDL'], 'detailed_test_type':..."
6040,73421364,\n\n VLDL 5-22 ...,"[{'test_name': ['VLDL'], 'detailed_test_type':..."


In [199]:
data2.loc[data2['검사결과'].isna() == True].count()

PT_NO               0
lab test results    0
검사결과                0
dtype: int64

In [200]:
data2['검사결과'][5]

[{'test_name': ['VLDL'],
  'detailed_test_type': ['none'],
  'test_results_value': ['5-22'],
  'test_result_unit': ['29'],
  'doctor_opinion': ['High']},
 {'test_name': ['IDL'],
  'detailed_test_type': ['C'],
  'test_results_value': ['11-22'],
  'test_result_unit': ['14'],
  'doctor_opinion': ['none']},
 {'test_name': ['IDL'],
  'detailed_test_type': ['B'],
  'test_results_value': ['5-15'],
  'test_result_unit': ['18'],
  'doctor_opinion': ['none']},
 {'test_name': ['IDL'],
  'detailed_test_type': ['A'],
  'test_results_value': ['8-25'],
  'test_result_unit': ['12'],
  'doctor_opinion': ['none']},
 {'test_name': ['LDL'],
  'detailed_test_type': ['1(large)'],
  'test_results_value': ['25-57'],
  'test_result_unit': ['19'],
  'doctor_opinion': ['Low']},
 {'test_name': ['LDL'],
  'detailed_test_type': ['2(large)'],
  'test_results_value': ['7-29'],
  'test_result_unit': ['19'],
  'doctor_opinion': ['none']},
 {'test_name': ['LDL'],
  'detailed_test_type': ['3(small)'],
  'test_results_val

In [ ]:
id_df = pd.DataFrame({'PT_NO' : [111]})


In [144]:
tmp_data = data2.loc[data2['PT_NO'] == 7745708, '검사결과'].reset_index(drop = True)
tmp_data = tmp_data[0]
tmp_data

[{'test_name': ['VLDL'],
  'detailed_test_type': ['none'],
  'test_results_value': ['5-22'],
  'test_result_unit': ['none'],
  'doctor_opinion': ['none']},
 {'test_name': ['IDL'],
  'detailed_test_type': ['none'],
  'test_results_value': ['C'],
  'test_result_unit': ['none'],
  'doctor_opinion': ['none']},
 {'test_name': ['IDL'],
  'detailed_test_type': ['none'],
  'test_results_value': ['B'],
  'test_result_unit': ['none'],
  'doctor_opinion': ['none']},
 {'test_name': ['IDL'],
  'detailed_test_type': ['none'],
  'test_results_value': ['A'],
  'test_result_unit': ['none'],
  'doctor_opinion': ['none']},
 {'test_name': ['LDL'],
  'detailed_test_type': ['none'],
  'test_results_value': ['1(large)'],
  'test_result_unit': ['none'],
  'doctor_opinion': ['none']},
 {'test_name': ['LDL'],
  'detailed_test_type': ['none'],
  'test_results_value': ['2(large)'],
  'test_result_unit': ['none'],
  'doctor_opinion': ['none']},
 {'test_name': ['LDL'],
  'detailed_test_type': ['none'],
  'test_resu

In [201]:
data2.loc[data2['PT_NO'] == 7768237, '검사결과']

3    [{'test_name': ['VLDL'], 'detailed_test_type':...
Name: 검사결과, dtype: object

In [202]:
id_list = data['PT_NO'].unique().tolist()

final_df = pd.DataFrame()
for id in id_list: 
    tmp_data = data2.loc[data2['PT_NO'] == id, '검사결과'].reset_index(drop = True)
    tmp_data = tmp_data[0]
    id_df = pd.DataFrame({'PT_NO' : [id]})
    df = pd.DataFrame()
    if tmp_data == None: 
        final_df = pd.concat([final_df, pd.DataFrame({'PT_NO' : [id], 'test_name' : None, 'detailed_test_type' : None, 'test_results_value' : None, 'test_result_unit' : None, 'doctor_opinion' : None})], axis = 0)
    else: 
        for i in range(len(tmp_data)): 
            k = pd.DataFrame.from_dict(data=tmp_data[i], orient = 'columns')
            df = pd.concat([df, k], axis = 0)
        df = pd.concat([id_df, df], axis = 1)
        final_df = pd.concat([final_df, df], axis = 0)
final_df.reset_index(drop = True, inplace = True)

In [204]:
final_df

,PT_NO,test_name,detailed_test_type,test_results_value,test_result_unit,doctor_opinion
0,7736588,VLDL,none,5-22,29,High
1,7736588,IDL,C,11-22,11,none
2,7736588,none,none,none,none,none
3,7736588,none,none,none,none,none
4,7736588,LDL,1(large),25-57,15,none
...,...,...,...,...,...,...
36442,76086258,LDL,4(small),0,none,none
36443,76086258,LDL,5(small),0,none,none
36444,76086258,LDL,6(small),0,none,none
36445,76086258,LDL,7(small),0,none,none


In [205]:
final_df.to_csv("C:\\Users\\admin\\Downloads\\ldlparticle_parsed.csv", encoding = 'euc-kr', index = False)

In [67]:
input_string = """
VLDL                     5-22           bb     High        

IDL   C                 11-22           cc 

      B                  5-15            dd

      A                  8-25            ee

LDL   1 (large)         25-57           ff

      2 (large)          7-29           gg

      3 (small)          0-4             hh

      4 (small)                          ii     High

      5 (small)                          jj

      6 (small)                          kk

      7 (small)                             ll        

HDL                     40-79           mm
"""


In [109]:
tmp = parse_lab_results(input_string)
id_df = pd.DataFrame({'PT_NO' : [111]})

df = pd.DataFrame()
for i in range(len(tmp)): 
    k = pd.DataFrame.from_dict(data=tmp[i], orient = 'columns')
    df = pd.concat([df, k], axis = 0)

df = pd.concat([id_df, df], axis = 1)
df

,PT_NO,test_name,detailed_test_type,test_results_value,test_result_unit,doctor_opinion
0,111,VLDL,none,5-22,bb,none
0,111,IDL,C,11-22,cc,none
0,111,IDL,B,5-15,dd,none
0,111,IDL,A,8-25,ee,none
0,111,LDL,1(large),25-57,ff,none
0,111,LDL,2(large),7-29,gg,none
0,111,LDL,3(small),0-4,hh,none
0,111,LDL,4(small),none,ii,none
0,111,LDL,5(small),none,jj,none
0,111,LDL,6(small),none,kk,none


-------------------

- transpose parsed data       
- left join with lab data & mortality

In [9]:
data = pd.read_csv("D:\\SNUlab\\ECG\\0. data\\ldlparticle_parsed_v2.csv", encoding = 'euc-kr')
data

,PT_NO,test_name,detailed_test_type,test_results_value,test_result_unit,doctor_opinion,Unnamed: 6,Unnamed: 7
0,7736588,VLDL,none,05월 22일,29,High,29,29.0
1,7736588,IDL,C,11월 22일,11,none,11,11.0
2,7736588,none,none,none,none,none,NaN,NaN
3,7736588,none,none,none,none,none,NaN,NaN
4,7736588,LDL,1(large),25-57,15,none,15,15.0
...,...,...,...,...,...,...,...,...
36442,76086258,LDL,4(small),0,none,none,0,0.0
36443,76086258,LDL,5(small),0,none,none,0,0.0
36444,76086258,LDL,6(small),0,none,none,0,0.0
36445,76086258,LDL,7(small),0,none,none,0,0.0


In [10]:
data['detailed_test_name'] = data['test_name'].astype(str) + ' ' + data['detailed_test_type'].astype(str)
data

,PT_NO,test_name,detailed_test_type,test_results_value,test_result_unit,doctor_opinion,Unnamed: 6,Unnamed: 7,detailed_test_name
0,7736588,VLDL,none,05월 22일,29,High,29,29.0,VLDL none
1,7736588,IDL,C,11월 22일,11,none,11,11.0,IDL C
2,7736588,none,none,none,none,none,NaN,NaN,none none
3,7736588,none,none,none,none,none,NaN,NaN,none none
4,7736588,LDL,1(large),25-57,15,none,15,15.0,LDL 1(large)
...,...,...,...,...,...,...,...,...,...
36442,76086258,LDL,4(small),0,none,none,0,0.0,LDL 4(small)
36443,76086258,LDL,5(small),0,none,none,0,0.0,LDL 5(small)
36444,76086258,LDL,6(small),0,none,none,0,0.0,LDL 6(small)
36445,76086258,LDL,7(small),0,none,none,0,0.0,LDL 7(small)


In [11]:
# drop if detailed test name is 'none none'
# 36447 -> 35255

data = data.loc[data['detailed_test_name'] != 'none none'].reset_index(drop = True)
data

,PT_NO,test_name,detailed_test_type,test_results_value,test_result_unit,doctor_opinion,Unnamed: 6,Unnamed: 7,detailed_test_name
0,7736588,VLDL,none,05월 22일,29,High,29,29.0,VLDL none
1,7736588,IDL,C,11월 22일,11,none,11,11.0,IDL C
2,7736588,LDL,1(large),25-57,15,none,15,15.0,LDL 1(large)
3,7736588,LDL,2(large),07월 29일,14,none,14,14.0,LDL 2(large)
4,7736588,LDL,3(small),0-4,6,none,6,6.0,LDL 3(small)
...,...,...,...,...,...,...,...,...,...
35250,76086258,LDL,4(small),0,none,none,0,0.0,LDL 4(small)
35251,76086258,LDL,5(small),0,none,none,0,0.0,LDL 5(small)
35252,76086258,LDL,6(small),0,none,none,0,0.0,LDL 6(small)
35253,76086258,LDL,7(small),0,none,none,0,0.0,LDL 7(small)


In [13]:
# select only columns needed 

data = data[['PT_NO', 'detailed_test_name', 'Unnamed: 7']]
data.rename(columns = {'Unnamed: 7' : 'value'}, inplace = True)
data

,PT_NO,detailed_test_name,value
0,7736588,VLDL none,29.0
1,7736588,IDL C,11.0
2,7736588,LDL 1(large),15.0
3,7736588,LDL 2(large),14.0
4,7736588,LDL 3(small),6.0
...,...,...,...
35250,76086258,LDL 4(small),0.0
35251,76086258,LDL 5(small),0.0
35252,76086258,LDL 6(small),0.0
35253,76086258,LDL 7(small),0.0


In [20]:
len(data['PT_NO'].unique())

4537

In [21]:
data = pd.DataFrame(data.pivot_table('value', ['PT_NO'], 'detailed_test_name'))
data

detailed_test_name,HDL none,IDL A,IDL B,IDL C,LDL 1(large),LDL 2(large),LDL 3(small),LDL 4(small),LDL 5(small),LDL 6(small),LDL 7(small),VLDL none
PT_NO,,,,,,,,,,,,
7736588,51.0,NaN,NaN,11.0,15.0,14.0,6.0,1.0,0.0,0.0,0.0,29.0
7745708,66.0,NaN,NaN,4.0,29.0,29.0,10.0,3.0,2.0,0.0,0.0,22.0
7749096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0
7768237,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0
7795741,29.0,13.0,16.0,22.0,19.0,14.0,2.0,0.0,0.0,0.0,0.0,31.0
...,...,...,...,...,...,...,...,...,...,...,...,...
71207632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.0
71332273,47.0,14.0,9.0,9.0,35.0,10.0,0.0,0.0,0.0,0.0,0.0,24.0
71604598,67.0,16.0,13.0,23.0,58.0,15.0,0.0,0.0,0.0,0.0,0.0,17.0


In [22]:
data.rename(columns = {'HDL none' : 'HDL', 
                       'VLDL none' : 'VLDL'}, inplace = True)
data

detailed_test_name,HDL,IDL A,IDL B,IDL C,LDL 1(large),LDL 2(large),LDL 3(small),LDL 4(small),LDL 5(small),LDL 6(small),LDL 7(small),VLDL
PT_NO,,,,,,,,,,,,
7736588,51.0,NaN,NaN,11.0,15.0,14.0,6.0,1.0,0.0,0.0,0.0,29.0
7745708,66.0,NaN,NaN,4.0,29.0,29.0,10.0,3.0,2.0,0.0,0.0,22.0
7749096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0
7768237,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0
7795741,29.0,13.0,16.0,22.0,19.0,14.0,2.0,0.0,0.0,0.0,0.0,31.0
...,...,...,...,...,...,...,...,...,...,...,...,...
71207632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.0
71332273,47.0,14.0,9.0,9.0,35.0,10.0,0.0,0.0,0.0,0.0,0.0,24.0
71604598,67.0,16.0,13.0,23.0,58.0,15.0,0.0,0.0,0.0,0.0,0.0,17.0


In [25]:
data.to_csv('D:\\SNUlab\\ECG\\0. data\\ldlparticle_pivot.csv', encoding = 'euc-kr')

In [23]:
# join with lab data

labdata = pd.read_csv('D:\\SNUlab\\ECG\\0. data\\ptlist+death+ecg+lab.csv')
labdata

,PT_NO,ecg_date,filepath,생년월일,성별,death_date,사망원인1,사망원인2,deathdiff,location,...,urine_PH,urine_RBC,urine_SG,urine_WBC,SBP,DBP,HR,sex,location_bool,death
0,44003751,2021-11-12,../dai/parsing_data/yoondid/SNUH/44003751_2021...,1971-08-14 00:00:00.000,M,NaN,NaN,NaN,NaN,HPC_ECG,...,5.0,4.24,1.020,3.80,119.0,62.0,NaN,1,0,0
1,45290178,2014-02-05,../dai/parsing_data/yoondid/SNUH/45290178_2014...,1952-04-11 00:00:00.000,M,NaN,NaN,NaN,NaN,HPC_ECG,...,7.0,4.65,1.014,4.50,110.0,63.0,NaN,1,0,0
2,37195227,2021-09-01,../dai/parsing_data/yoondid/SNUH/37195227_2021...,1961-12-10 00:00:00.000,M,NaN,NaN,NaN,NaN,HPC_ECG,...,7.5,4.73,1.012,4.52,NaN,NaN,NaN,1,0,0
3,33298100,2004-10-19,../dai/parsing_data/yoondid/SNUH/33298100_2004...,1945-01-14 00:00:00.000,M,NaN,NaN,NaN,NaN,HPC_ECG,...,7.0,3.72,1.022,5.22,97.0,60.0,87.0,1,0,0
4,47050888,2015-07-02,../dai/parsing_data/yoondid/SNUH/47050888_2015...,1966-02-14 00:00:00.000,F,NaN,NaN,NaN,NaN,HPC_ECG,...,5.0,5.13,1.023,6.70,110.0,70.0,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183062,39302432,2021-10-22,../dai/parsing_data/yoondid3/KANGNAM/39302432_...,1968-04-23 00:00:00.000,F,NaN,NaN,NaN,NaN,ECG_KANGNAM,...,6.0,3.97,1.034,6.80,82.0,54.0,58.0,0,1,0
183063,43741339,2021-09-27,../dai/parsing_data/yoondid3/KANGNAM/43741339_...,1965-12-05 00:00:00.000,M,NaN,NaN,NaN,NaN,ECG_KANGNAM,...,5.5,5.11,1.028,5.10,NaN,NaN,NaN,1,1,0
183064,44727949,2013-09-11,../dai/parsing_data/yoondid3/KANGNAM/44727949_...,1961-08-15 00:00:00.000,M,2015-07-22,C34,NaN,679.0,ECG_KANGNAM,...,7.5,3.55,1.008,4.60,NaN,NaN,NaN,1,1,1
183065,39236984,2015-04-20,../dai/parsing_data/yoondid3/KANGNAM/39236984_...,1956-11-10 00:00:00.000,F,NaN,NaN,NaN,NaN,ECG_KANGNAM,...,6.0,4.82,1.026,6.20,NaN,NaN,NaN,0,1,0


In [65]:
# inner join with ldl data

joineddata = pd.merge(data, labdata, on='PT_NO', how='inner')
joineddata

,PT_NO,HDL_x,IDL A,IDL B,IDL C,LDL 1(large),LDL 2(large),LDL 3(small),LDL 4(small),LDL 5(small),...,urine_PH,urine_RBC,urine_SG,urine_WBC,SBP,DBP,HR,sex,location_bool,death
0,10372074,57.0,14.0,11.0,16.0,50.0,37.0,3.0,0.0,0.0,...,7.5,4.73,1.009,5.60,NaN,NaN,67.0,1,1,0
1,10466513,40.0,NaN,NaN,4.0,14.0,18.0,17.0,8.0,2.0,...,7.0,3.89,1.018,2.75,108.0,63.0,65.0,1,1,0
2,10483657,43.0,NaN,NaN,9.0,35.0,30.0,8.0,0.0,0.0,...,7.0,4.83,1.019,8.60,122.0,69.0,87.0,0,1,0
3,10556771,30.0,19.0,16.0,18.0,35.0,28.0,6.0,0.0,0.0,...,5.0,4.56,1.022,5.50,156.0,76.0,NaN,1,1,0
4,10594076,41.0,12.0,13.0,6.0,32.0,47.0,12.0,0.0,0.0,...,5.0,4.17,1.013,4.40,NaN,NaN,NaN,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1447,55836160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.5,4.68,1.012,6.74,NaN,NaN,NaN,1,1,0
1448,70330553,50.0,NaN,NaN,11.0,31.0,16.0,3.0,0.0,0.0,...,5.0,4.52,1.023,5.50,90.0,60.0,NaN,1,1,0
1449,71332273,47.0,14.0,9.0,9.0,35.0,10.0,0.0,0.0,0.0,...,7.0,5.22,1.017,3.80,NaN,NaN,NaN,0,1,0
1450,73421364,39.0,13.0,7.0,20.0,35.0,7.0,0.0,0.0,0.0,...,5.0,4.62,1.026,7.50,NaN,NaN,NaN,0,1,0


In [66]:
len(joineddata['PT_NO'].unique())

1452

In [67]:
joineddata.columns

Index(['PT_NO', 'HDL_x', 'IDL A', 'IDL B', 'IDL C', 'LDL 1(large)',
       'LDL 2(large)', 'LDL 3(small)', 'LDL 4(small)', 'LDL 5(small)',
       'LDL 6(small)', 'LDL 7(small)', 'VLDL', 'ecg_date', 'filepath', '생년월일',
       '성별', 'death_date', '사망원인1', '사망원인2', 'deathdiff', 'location', 'age',
       'ALT', 'AST', 'BUN', 'Ca', 'Cl(serum)', 'HDL_y', 'Hb', 'LDL',
       'Na(serum)', 'P', 'PLT', 'TG', 'albumin', 'chol', 'glucose', 'hct',
       'k(serum)', 'protein(total)', 'totalBilirubin', 'uric_acid', 'urine_PH',
       'urine_RBC', 'urine_SG', 'urine_WBC', 'SBP', 'DBP', 'HR', 'sex',
       'location_bool', 'death'],
      dtype='object')

In [68]:
joineddata = joineddata.loc[joineddata['LDL 1(large)'].isna() == False].reset_index(drop = True)
joineddata

,PT_NO,HDL_x,IDL A,IDL B,IDL C,LDL 1(large),LDL 2(large),LDL 3(small),LDL 4(small),LDL 5(small),...,urine_PH,urine_RBC,urine_SG,urine_WBC,SBP,DBP,HR,sex,location_bool,death
0,10372074,57.0,14.0,11.0,16.0,50.0,37.0,3.0,0.0,0.0,...,7.5,4.73,1.009,5.60,NaN,NaN,67.0,1,1,0
1,10466513,40.0,NaN,NaN,4.0,14.0,18.0,17.0,8.0,2.0,...,7.0,3.89,1.018,2.75,108.0,63.0,65.0,1,1,0
2,10483657,43.0,NaN,NaN,9.0,35.0,30.0,8.0,0.0,0.0,...,7.0,4.83,1.019,8.60,122.0,69.0,87.0,0,1,0
3,10556771,30.0,19.0,16.0,18.0,35.0,28.0,6.0,0.0,0.0,...,5.0,4.56,1.022,5.50,156.0,76.0,NaN,1,1,0
4,10594076,41.0,12.0,13.0,6.0,32.0,47.0,12.0,0.0,0.0,...,5.0,4.17,1.013,4.40,NaN,NaN,NaN,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087,48658696,45.0,12.0,10.0,11.0,39.0,34.0,10.0,2.0,0.0,...,7.0,4.43,1.016,4.00,NaN,NaN,NaN,1,1,0
1088,70330553,50.0,NaN,NaN,11.0,31.0,16.0,3.0,0.0,0.0,...,5.0,4.52,1.023,5.50,90.0,60.0,NaN,1,1,0
1089,71332273,47.0,14.0,9.0,9.0,35.0,10.0,0.0,0.0,0.0,...,7.0,5.22,1.017,3.80,NaN,NaN,NaN,0,1,0
1090,73421364,39.0,13.0,7.0,20.0,35.0,7.0,0.0,0.0,0.0,...,5.0,4.62,1.026,7.50,NaN,NaN,NaN,0,1,0


In [69]:
for col in joineddata.columns: 
    print(col, joineddata[col].isna().sum())

PT_NO 0
HDL_x 0
IDL A 383
IDL B 383
IDL C 0
LDL 1(large) 0
LDL 2(large) 0
LDL 3(small) 0
LDL 4(small) 0
LDL 5(small) 0
LDL 6(small) 0
LDL 7(small) 0
VLDL 0
ecg_date 0
filepath 0
생년월일 0
성별 0
death_date 1080
사망원인1 1080
사망원인2 1092
deathdiff 1080
location 0
age 0
ALT 23
AST 26
BUN 15
Ca 16
Cl(serum) 39
HDL_y 471
Hb 16
LDL 143
Na(serum) 39
P 15
PLT 18
TG 23
albumin 132
chol 15
glucose 20
hct 18
k(serum) 39
protein(total) 15
totalBilirubin 16
uric_acid 15
urine_PH 31
urine_RBC 18
urine_SG 31
urine_WBC 18
SBP 528
DBP 528
HR 787
sex 0
location_bool 0
death 0


In [84]:
len(joineddata.columns)

54

In [83]:
joineddata.loc[joineddata['death'] == 1]

,PT_NO,HDL_x,IDL A,IDL B,IDL C,LDL1_large,LDL2_large,LDL3_small,LDL4_small,LDL5_small,...,urine_RBC,urine_SG,urine_WBC,SBP,DBP,HR,sex,location_bool,death,TIME
39,15548775,42.0,17.0,12.0,21.0,28.0,12.0,0.0,0.0,0.0,...,5.25,1.015,4.90,NaN,NaN,NaN,1,1,1,32926
50,16675113,22.0,NaN,NaN,3.0,13.0,13.0,5.0,1.0,0.0,...,4.50,1.024,5.40,NaN,NaN,NaN,1,0,1,25811
87,19234092,23.0,21.0,23.0,24.0,25.0,21.0,16.0,6.0,0.0,...,4.71,1.018,6.00,140.0,90.0,78.0,1,1,1,26640
203,25452345,54.0,7.0,6.0,14.0,20.0,6.0,1.0,0.0,0.0,...,4.08,1.028,4.69,NaN,NaN,NaN,1,1,1,27412
206,25643817,38.0,20.0,15.0,13.0,30.0,25.0,5.0,0.0,0.0,...,5.31,1.026,6.30,97.0,68.0,NaN,1,1,1,25039
263,28337368,49.0,12.0,6.0,2.0,38.0,29.0,3.0,0.0,0.0,...,5.29,1.023,6.25,NaN,NaN,NaN,1,0,1,30314
275,28651653,48.0,NaN,NaN,11.0,20.0,21.0,14.0,3.0,0.0,...,4.62,1.021,7.30,103.0,69.0,70.0,1,1,1,25502
300,29752287,46.0,15.0,10.0,14.0,52.0,12.0,0.0,0.0,0.0,...,4.33,1.018,4.70,NaN,NaN,NaN,0,0,1,27718
366,32105173,29.0,12.0,18.0,23.0,21.0,27.0,8.0,0.0,0.0,...,4.37,1.008,4.61,118.0,65.0,81.0,1,1,1,24336
520,33523673,64.0,NaN,NaN,7.0,36.0,25.0,4.0,1.0,0.0,...,5.47,1.028,8.15,NaN,NaN,NaN,1,1,1,27368


In [71]:
# datetime 형식으로 변경

joineddata['ecg_date'] = pd.to_datetime(joineddata['ecg_date'])
joineddata['death_date'] = pd.to_datetime(joineddata['death_date'])
joineddata['생년월일'] = pd.to_datetime(joineddata['생년월일'])

In [72]:
# TIME 변수 생성

joineddata.loc[joineddata['death'] == 1, 'TIME'] = joineddata['death_date'] - joineddata['생년월일']
joineddata.loc[joineddata['death'] == 0, 'TIME'] = joineddata['ecg_date'] - joineddata['생년월일']

# TIME 변수 int 형식으로 변경

joineddata['TIME'] = joineddata['TIME'].dt.days

In [73]:
len(joineddata['TIME'].unique())

1040

In [74]:
joineddata.loc[joineddata['TIME'] < 0]

,PT_NO,HDL_x,IDL A,IDL B,IDL C,LDL 1(large),LDL 2(large),LDL 3(small),LDL 4(small),LDL 5(small),...,urine_RBC,urine_SG,urine_WBC,SBP,DBP,HR,sex,location_bool,death,TIME


In [75]:
for col in joineddata.columns: 
    print(col, joineddata[col].isna().sum())

PT_NO 0
HDL_x 0
IDL A 383
IDL B 383
IDL C 0
LDL 1(large) 0
LDL 2(large) 0
LDL 3(small) 0
LDL 4(small) 0
LDL 5(small) 0
LDL 6(small) 0
LDL 7(small) 0
VLDL 0
ecg_date 0
filepath 0
생년월일 0
성별 0
death_date 1080
사망원인1 1080
사망원인2 1092
deathdiff 1080
location 0
age 0
ALT 23
AST 26
BUN 15
Ca 16
Cl(serum) 39
HDL_y 471
Hb 16
LDL 143
Na(serum) 39
P 15
PLT 18
TG 23
albumin 132
chol 15
glucose 20
hct 18
k(serum) 39
protein(total) 15
totalBilirubin 16
uric_acid 15
urine_PH 31
urine_RBC 18
urine_SG 31
urine_WBC 18
SBP 528
DBP 528
HR 787
sex 0
location_bool 0
death 0
TIME 0


In [76]:
joineddata.rename(columns = {'LDL 1(large)' : 'LDL1_large', 
'LDL 2(large)' : 'LDL2_large',
'LDL 3(small)' : 'LDL3_small',
'LDL 4(small)' : 'LDL4_small',
'LDL 5(small)' : 'LDL5_small',
'LDL 6(small)' : 'LDL6_small',
'LDL 7(small)' : 'LDL7_small', }, inplace = True)

In [80]:
joineddata.describe()

,PT_NO,HDL_x,IDL A,IDL B,IDL C,LDL1_large,LDL2_large,LDL3_small,LDL4_small,LDL5_small,...,urine_RBC,urine_SG,urine_WBC,SBP,DBP,HR,sex,location_bool,death,TIME
count,1.092000e+03,1092.000000,709.000000,709.000000,1092.000000,1092.000000,1092.000000,1092.000000,1092.000000,1092.000000,...,1074.000000,1061.000000,1074.000000,564.000000,564.000000,305.000000,1092.000000,1092.000000,1092.000000,1092.000000
mean,3.213930e+07,43.055861,13.928068,11.758815,138.054029,32.002747,23.911172,8.287546,2.424908,0.464286,...,4.674590,1.021328,5.524767,110.907801,70.813830,76.245902,0.818681,0.876374,0.010989,22955.237179
std,7.563013e+06,12.601616,5.348495,4.644077,2368.961693,12.360718,10.928917,7.765804,4.470172,1.702860,...,0.464381,0.006998,1.585134,14.978327,9.999951,12.476367,0.385459,0.329306,0.104299,4073.761145
min,1.037207e+07,0.000000,3.000000,2.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,...,3.140000,1.000000,2.000000,3.000000,44.000000,49.000000,0.000000,0.000000,0.000000,7013.000000
25%,2.859581e+07,34.000000,10.000000,9.000000,9.000000,23.000000,16.000000,2.000000,0.000000,0.000000,...,4.340000,1.017000,4.400000,100.000000,64.000000,68.000000,1.000000,1.000000,0.000000,20360.000000
50%,3.368655e+07,41.000000,13.000000,11.000000,13.000000,30.000000,23.000000,5.000000,0.000000,0.000000,...,4.640000,1.022000,5.230000,110.000000,71.000000,76.000000,1.000000,1.000000,0.000000,23158.000000
75%,3.617045e+07,50.000000,18.000000,14.000000,17.000000,39.000000,31.000000,14.000000,3.000000,0.000000,...,4.987500,1.025000,6.375000,120.000000,78.000000,84.000000,1.000000,1.000000,0.000000,25718.750000
max,7.608626e+07,86.000000,43.000000,34.000000,45252.000000,109.000000,66.000000,35.000000,27.000000,26.000000,...,6.630000,1.050000,16.650000,158.000000,110.000000,120.000000,1.000000,1.000000,1.000000,35180.000000


In [77]:
joineddata.to_csv('D:\\SNUlab\\ECG\\0. data\\ldlparticle_labdatajoined.csv', encoding = 'euc-kr', index = False)